In [1]:
import pandas as pd
import numpy as np
import os
import pymaltspro as pmp

In [3]:
data_tables_directory = './cgm_analysis/RT-CGM Randomized Clinical Trial/DataTables'
control_group_cgm = pd.concat([pd.read_csv(data_tables_directory + '/tblADataRTCGM_Unblinded_ControlGroup_' + str(i) + '.csv') for i in range(1, 6)], axis = 0)
treated_group_cgm = pd.concat([pd.read_csv(data_tables_directory + '/tblADataRTCGM_Unblinded_RTCGMGroup_' + str(i) + '.csv') for i in range(1, 13)], axis = 0)
cgm_data = pd.concat([control_group_cgm, treated_group_cgm], axis = 0)

In [9]:
# n_min_samples = cgm_data.dropna().groupby('PtID')['Glucose'].count().min()
qtl_df = cgm_data.groupby('PtID')[['Glucose']].quantile(q = np.linspace(0, 1, num = n_min_samples))
# qtl_df = qtl_df.reset_index().rename(columns = {'level_1' : 'quantile'}).pivot_table(index = 'PtID', columns = 'quantile', values = 'Glucose').reset_index()
# qtl_df = qtl_df.set_index('PtID')


964

In [ ]:
demographic_data = pd.read_csv(data_tables_directory + '/tblAPtSummary.csv')
demographic_data['Treatment'] = (demographic_data['TxGroup'] == 'RT-CGM').astype(int).values
demo_cols = ['PtID', 'Gender', 'AgeAsOfRandDt', 'Race', 'DurDiabetes', 'EduCareGvrPEdu', 'Treatment'] #list(demographic_data.columns)
demo_disc_cols = ['Gender', 'Race', 'EduCareGvrPEdu']
demographic_data = demographic_data.loc[demographic_data.PtID.isin(qtl_df.index.unique())].set_index('PtID')

In [ ]:
# demographic_data.columns
# demographic_data[['PtID', 'Gender', 'AgeAsOfRandDt', 'Race', 'DurDiabetes', 'EduCareGvrPEdu', 'Treatment']]

## Dist Outcome MALTS

In [ ]:
np.random.seed(999)
n_units = demographic_data.index.shape[0]
train_indexes = np.random.choice(demographic_data.index.values, size = int(0.2 * n_units), replace = False)
est_indexes = list(set(demographic_data.index.values) - set(train_indexes))
X_train = demographic_data.loc[train_indexes, :].reset_index()
X_est = demographic_data.loc[est_indexes, :].reset_index()
y_train = qtl_df.loc[train_indexes, :].to_numpy()
y_est = qtl_df.loc[est_indexes, :].to_numpy()

In [ ]:
# run MALTSPro
maltspro = pmp.pymaltspro(X = X_train,
                            y = y_train, 
                            treatment = 'Treatment', 
                            discrete = demo_disc_cols,
                            C = 0.001,
                            k = 10)


AttributeError: 'DataFrame' object has no attribute 'to_numpy'